<a href="https://colab.research.google.com/github/sharky1746/20-1-AI-Video-Processing/blob/main/%EC%9D%B4%EC%83%81%ED%9B%88_%EA%B8%B0%EB%A7%903.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 95kB/s 
     |████████████████████████████████| 3.2MB 35.3MB/s 
     |████████████████████████████████| 491kB 25.1MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_valid, y_valid = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

########## Parameters
epochs = 10
batch_size = 100

########## Functions
def conv2d(input, filters, kernel_size, strides=1, activation=tf.nn.relu, padding='SAME', name='conv'):
  with tf.variable_scope(name, reuse=False):
    out = tf.layers.conv2d(input, filters=filters, kernel_size=kernel_size,
                           strides=strides, padding=padding, activation=activation, name=name)
    return out

def dense(input, unit, activation=tf.nn.relu, name='dense'):
  with tf.variable_scope(name, reuse=False):
    out = tf.layers.dense(input, unit, activation=activation, name=name)
  return out

def max_pool(input, k, s, name='pool'):
  out = tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='SAME', name=name)
  return out

########## FC ##########
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

fc1 = dense(x, 60, name='fc1')

########## PROJECT 2
fc2 = dense(fc1, 30, name='fc2')

fc_out = dense(fc2, 10, activation=tf.nn.softmax, name='fc_out')

cross_entropy_FC = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=fc_out))
optim_FC = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy_FC)

correct_prediction_FC = tf.equal(tf.argmax(y, 1), tf.argmax(fc_out, 1))
accuracy_FC = tf.reduce_mean(tf.cast(correct_prediction_FC, tf.float32))

########## CNN ##########
x_image = tf.reshape(x, [-1, 28, 28, 1])

conv1 = conv2d(x_image, 96, 3, 1, tf.nn.relu, 'SAME', name='conv1')
pool1 = max_pool(conv1, 2, 2, name='pool1')

########## PROJECT 3
conv2 = conv2d(pool1, 96, 7, 1, tf.nn.relu, 'SAME', name='conv2')
pool2 = max_pool(conv2, 8, 2, name='pool2')

########## PROJECT 4
# conv3 =
# pool3 =
# conv4 =
# pool4 =
# conv5 =
# pool5 =

########## PROJECT 5
# short cut (pool1 out & conv5 in)

cnn_out = tf.reshape(pool2, [batch_size, tf.size(pool2[0])])
cnn_out = dense(cnn_out, 10, activation=tf.nn.softmax, name='cnn_out')

cross_entropy_CNN = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=cnn_out))
optim_CNN = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy_CNN)

correct_prediction_CNN = tf.equal(tf.argmax(cnn_out,1), tf.argmax(y,1))
accuracy_CNN = tf.reduce_mean(tf.cast(correct_prediction_CNN, tf.float32))

########## SESSION ##########
init = tf.initialize_all_variables()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False,
                                      gpu_options=tf.GPUOptions(allow_growth=True, visible_device_list='0'))) as sess:
  sess.run(init)

  print('Training...')
  for i in range(epochs):
    s = np.random.permutation(len(X_train))
    X_train = X_train[s]
    y_train = y_train[s]
    accCNN_total, accFC_total = 0, 0
    for offset in range(0, len(X_train), batch_size):
      end = offset + batch_size
      batch_x, batch_y = X_train[offset:end], y_train[offset:end]
      _, accFC = sess.run([optim_FC, accuracy_FC], feed_dict={x: batch_x, y: batch_y})
      _, accCNN = sess.run([optim_CNN, accuracy_CNN], feed_dict={x: batch_x, y: batch_y})
      accFC_total += accFC
      accCNN_total += accCNN
    accFC_total /= (len(X_train) / batch_size)
    accCNN_total /= (len(X_train) / batch_size)
    print("[TRAIN] epoch: " + str(i) + ", FC Accuracy= " + "{:.4f}".format(accFC_total) +
          ", CNN Accuracy= " + "{:.4f}".format(accCNN_total))

  print("Optimization Finished!")
  for offset in range(0, len(X_test), batch_size):
    end = offset + batch_size
    batch_x, batch_y = X_test[offset:end], y_test[offset:end]
    _, acc = sess.run([optim_FC, accuracy_FC], feed_dict={x: batch_x, y: batch_y})
    _, accCNN = sess.run([optim_CNN, accuracy_CNN], feed_dict={x: batch_x, y: batch_y})
    accFC_total += accFC
    accCNN_total += accCNN
  accFC_total /= (len(X_test) / batch_size)
  accCNN_total /= (len(X_test) / batch_size)
  print("[TEST] FC Accuracy= " + "{:.4f}".format(accFC_total) + ", CNN Accuracy= " + "{:.4f}".format(accCNN_total))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr